In [1]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam

# Define the model-building function for Keras Tuner
def build_model(hp):
    model = Sequential()
    
    # Tune the number of LSTM units in the first LSTM layer
    units = hp.Int("units", min_value=20, max_value=100, step=20)
    model.add(Bidirectional(LSTM(units=units, return_sequences=True)))
    model.add(Dropout(0.2))

    # Tune the number of LSTM units in the second LSTM layer
    units2 = hp.Int("units2", min_value=20, max_value=100, step=20)
    model.add(Bidirectional(LSTM(units=units2)))
    model.add(Dropout(0.2))
    
    # Output layer
    model.add(Dense(1))
    
    # Tune the learning rate
    learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4, 1e-5])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mean_squared_error")
    
    return model

# Set up the tuner with a specific objective and tuning range
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=20,  # Number of different hyperparameter configurations to try
    executions_per_trial=2,  # Number of models to train per configuration to average out variability
    directory="hyperparameter_tuning",
    project_name="lstm_tuning"
)

# Run the hyperparameter search
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

# Get the best model and hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best number of units in first LSTM layer: {best_hps.get('units')}")
print(f"Best number of units in second LSTM layer: {best_hps.get('units2')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")

# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32)

# Evaluate the best model
loss = best_model.evaluate(X_test, y_test)
print(f"Test loss of best model: {loss}")

ModuleNotFoundError: No module named 'keras_tuner'